# Implementation and Evaluation of Serverless Applications in the AWS Cloud Environment

## Structure 1 - Serverless Storage and Connector

## Structure 2 - Serverless WebApp

A web application using HTTP API will be deployed. This API includes various routes, such as `options`, `get`, `put`, and `delete`, corresponding to different paths. 

The setup includes a Lambda function and a DynamoDB table. To make it accessible to users, a static website hosted on Amazon S3 will be deployed. This static website will serve as the front-end interface for users, who can interact with the API via their web browsers.

The static website will contain a form where users can enter and submit information. The form will communicate with the public endpoint of the HTTP API, which will pass the data to the Lambda function. The Lambda function will process the request, and the processed data will be stored in a DynamoDB table.

1. A user interacts with the static website hosted on S3.
2. The website communicates with the HTTP API.
3. The API sends the request to the Lambda function.
4. The Lambda function processes the request and stores the result in the DynamoDB table.

The form that's generated will also be populated with information from the table, so it will always be returned back and viewable.

<img src="figs/serverless_webapp.png"  style="width:85%;"/>

### Create a Stack

On Amazon Cloud Formation, we create a Stack from an uploaded template `code/aws-backend-creation-cftemplate.yaml`

Stack name: `WebApp`

From this template, we can visualize the Application Composer canvas:

<img src="figs/application_composer_canvas.png"  style="width:85%;"/>

We need to check the field "I acknowledge that AWS CloudFormation might create IAM resources".

Click `Submit`.

### Create an S3 Bucket

On S3, create a bucket.

Bucket name: `frontend-website-0`

After creating the bucket, click on it and select "Properties". On "Static website hosting", click on "Edit". Select `Enable`.

Index document: `Index.html`.

Save changes.

Click on "Permissions". On "Block public access (bucket settings)", click on "Edit". Disable `Block all public access`.

Save changes.

Then Edit "Bucket policy". Insert the following code policy:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::frontend-website-0/*"
        }
    ]
}

```

The Bucket ARN (`arn:aws:s3:::frontend-website-0`) is the field in "Resource" plus "/*".

The bucket policy, written in JSON, provides access to the objects stored in the bucket. Now we have a bucket policy, which allows GetObjects (that is, we can have public reads of objects within the bucket).

Go to "Cross-origin resource sharing (CORS)" and edit it. Insert this:

```json
[
    {
        "AllowedHeaders": [
            "*"
        ],
        "AllowedMethods": [
            "GET"
        ],
        "AllowedOrigins": [
            "*"
        ],
        "ExposeHeaders": []
    }
]
```

The CORS configuration, written in JSON, defines a way for client web applications that are loaded in one domain to interact with resources in a different domain. This CORS policy allows any headers, the `GET` method, and any origins.





### Configure API Gateway

Go to "API Gateway". You can see that Cloud Formation created an API called `items-api`. Click on it and go to "Stages". Click on "Create".

Name: `prod`.

Click on "create".

Go to "Integrations" > "Manage integrations" > "Create".

Attach this integration to a route: `$default`

Integration type: `Lambda function`

AWS Region: `us-east-1` (same for every region asked before)

Lambda function: `arn:aws:lambda:us-east-1:093511299840:function:WebApp-ItemLambdaFunction-VwHfJ4KyYg2b`

Click on "Create".

Copy the "Integration ID": `dhp9nre`.

Click on "Routes" > "Create a route". We'll create different routes for different methods. In "Route and method", add the following:

`GET` -> `/items`

`PUT` -> `/items`

`OPTIONS` -> `/items`

`GET` -> `/items/{id}`

`DELETE` -> `/items/{id}`

`OPTIONS` -> `/items/{id}`

So we have:

```asciidoc
$default
├── /items
│   ├── PUT
│   ├── OPTIONS
│   ├── GET
│   └── /{id}
│       ├── OPTIONS
│       ├── GET
│       └── DELETE
```

Then we select each one of these, choose "Attach integration" > "Choose an existing integration". Then choose the one created before (remember Integration ID) and click on "Attach integration"

<img src="figs/routes_for_items_api.png"  style="width:15%;"/>

Go to "CORS" > "Configure".

Access-Control-Allow-Origin: `https://frontend-website-0.s3.amazonaws.com`

Access-Control-Allow-Headers: `*`

Access-Control-Allow-Methods: `*`

Access-Control-Expose-Headers: `*`

Access-Control-Max-Age: `96400`

Access-Control-Allow-Credentials: `yes`

Click on "Save". Click on "Deploy".

Stage: `prod`

In `API Gateway\APIs\items-api\Stages`, we can find `prod`'s invoke URL: `https://ca75u89v9f.execute-api.us-east-1.amazonaws.com/prod`

### Client-side code

On our `code/client/src/config.ts` file, we need to have the invoke URL:

```typescript
const apiId = 'ca75u89v9f'
export const apiEndpoint = `https://${apiId}.execute-api.us-east-1.amazonaws.com/prod`
```

Open a terminal.

```
PS C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client> npm install

```